In [2]:
%pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/b6/cd/0c4ae4c85f15b230085efa2f12c1c96c7a0f7bef6f70e3a614ea5b334be6/ultralytics-8.3.240-py3-none-any.whl.metadata
  Using cached ultralytics-8.3.240-py3-none-any.whl.metadata (37 kB)
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/fa/80/eb88edc2e2b11cd2dd2e56f1c80b5784d11d6e6b7f04a1145df64df40065/opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for requests>=2.23.0 from https://files.pythonhosted.org/packages/1e/db/4254e3eabe8020b458f1a747140d32277ec7a271daf1d235b70dc0b4e6e3/requests-2.32.5-py3-none-any.whl.metadata
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/b1/1a/64f5769025db846a82567fa5b7d21dba45

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import subprocess
import sys

# ==========================================
# 1. 설정
# ==========================================
# [중요] Roboflow에서 'YOLO v5 PyTorch' 포맷으로 다운받은 폴더명
dataset_folder_name = "box_dataset_2" 

# 현재 경로 확인
current_dir = os.getcwd()
data_yaml_path = os.path.join(current_dir, dataset_folder_name, "data.yaml")
yolov5_dir = os.path.join(current_dir, "yolov5")

print(f"데이터셋 경로: {data_yaml_path}")

# ==========================================
# 2. YOLOv5 환경 준비 (자동 설치)
# ==========================================
# yolov5 폴더가 없으면 깃허브에서 다운로드(Clone) 합니다.
if not os.path.exists(yolov5_dir):
    print(">>> YOLOv5 설치 폴더가 없습니다. GitHub에서 다운로드합니다...")
    subprocess.run(["git", "clone", "https://github.com/ultralytics/yolov5"], check=True)
    
    # 필수 라이브러리 설치 (requirements.txt)
    print(">>> 필수 라이브러리를 설치합니다...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", os.path.join(yolov5_dir, "requirements.txt")], check=True)

# ==========================================
# 3. 모델 학습 (YOLOv5s)
# ==========================================
print(">>> YOLOv5s (Small) 학습을 시작합니다...")

# 학습 명령어 구성
# python yolov5/train.py --img 224 --batch 8 --epochs 50 --data "..." --weights yolov5s.pt
train_cmd = [
    sys.executable, os.path.join(yolov5_dir, "train.py"), # train.py 실행
    "--img", "224",             # 이미지 크기 (속도를 위해 224 유지)
    "--batch", "8",             # 배치 사이즈
    "--epochs", "50",           # 학습 횟수
    "--data", data_yaml_path,   # 데이터셋 경로 (data.yaml)
    "--weights", "yolov5s.pt",  # [중요] Jetson Nano용 추천 모델 (Small)
    "--name", "cube_yolov5_result" # 결과 저장 폴더 이름
]

# 명령어 실행
try:
    subprocess.run(train_cmd, check=True)
    print("\n>>> 학습 완료! 결과는 'yolov5/runs/train/cube_yolov5_result' 폴더에 저장됩니다.")
    print(">>> 'best.pt' 파일을 Jetson Nano로 옮기시면 됩니다.")
except subprocess.CalledProcessError as e:
    print(f"\n>>> 학습 중 오류가 발생했습니다: {e}")

데이터셋 경로: c:\Users\user\OneDrive\바탕 화면\jupyter_ex\RobotPal-jupyter\box_dataset_2\data.yaml
>>> YOLOv5s (Small) 학습을 시작합니다...

>>> 학습 중 오류가 발생했습니다: Command '['c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\python.exe', 'c:\\Users\\user\\OneDrive\\바탕 화면\\jupyter_ex\\RobotPal-jupyter\\yolov5\\train.py', '--img', '224', '--batch', '8', '--epochs', '50', '--data', 'c:\\Users\\user\\OneDrive\\바탕 화면\\jupyter_ex\\RobotPal-jupyter\\box_dataset_2\\data.yaml', '--weights', 'yolov5s.pt', '--name', 'cube_yolov5_result']' returned non-zero exit status 1.


In [ ]:
# from ultralytics import YOLO
# import os

# # ==========================================
# # 1. 설정
# # ==========================================
# # 다운로드 받은 OBB 데이터셋 폴더 이름으로 변경해주세요
# # (Roboflow 등에서 다운받으면 폴더명이 다를 수 있으니 꼭 확인하세요)
# dataset_folder_name = "box_dataset" 

# # data.yaml 파일의 절대 경로를 자동으로 찾습니다
# current_dir = os.getcwd()
# data_yaml_path = os.path.join(current_dir, dataset_folder_name, "data.yaml")

# print(f"데이터셋 경로: {data_yaml_path}")

# # ==========================================
# # 2. 모델 생성 및 학습 (OBB 모드)
# # ==========================================
# # 중요: OBB 학습을 위해 'yolov8n-obb.pt' 모델을 로드합니다.
# # 처음 실행 시 자동으로 다운로드 됩니다.
# model = YOLO("yolov8n-obb.pt") 

# print(">>> OBB(회전 박스) 학습을 시작합니다... (시간이 좀 걸립니다)")

# # 학습 시작!
# # 데이터셋이 OBB 형식이라면 모델이 자동으로 감지하여 Oriented Bounding Box 학습을 수행합니다.
# model.train(
#     data=data_yaml_path, 
#     # epochs=50, 
#     imgsz=224,       # 416보다 훨씬 정밀하게 각도를 계산합니다.
#     batch=8,         # 해상도가 높아지면 메모리를 많이 먹으므로 배치를 조금 줄였습니다.
#     device='cpu'
# )

# print(">>> 학습 완료!")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\user\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
데이터셋 경로: c:\Users\user\OneDrive\바탕 화면\jupyter_ex\RobotPal-jupyter\box_dataset\data.yaml
>>> OBB(회전 박스) 학습을 시작합니다... (시간이 좀 걸립니다)
Ultralytics 8.3.240  Python-3.12.0 torch-2.9.1+cpu CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\user\OneDrive\ \jupyter_ex\RobotPal-jupyter\box_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=N